In [1]:
from deeppavlov import build_model, configs
model = build_model(configs.squad.squad, download=True)

2019-04-10 11:44:33.230 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-04-10 11:44:34.943 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /embeddings/wiki-news-300d-1M.vec.md5 HTTP/1.1" 200 56
2019-04-10 11:45:11.397 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec download because of matching hashes
2019-04-10 11:45:11.411 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-04-10 11:45:11.882 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /embeddings/wiki-news-300d-1M-char.vec.md5 HTTP/1.1" 200 61
2019-04-10 11:45:12.28 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec download because 

In [2]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re
import requests
from collections import Counter
import bs4
import math
import urllib3

In [3]:
def extract_context(url):
    html = requests.get(url).content
    unicode_str = html.decode("utf8")
    encoded_str = unicode_str.encode("ascii",'ignore')
    news_soup = BeautifulSoup(encoded_str, "html.parser")
    p_text = news_soup.find_all('p')
    context = " "
    for p in p_text:
        context = context + p.text
    li_text = news_soup.find_all('li')
    for li in li_text:
        context = context + li.text
    return context

In [4]:
def text_to_vector(text):
    WORD = re.compile(r'\w+')
    words = WORD.findall(text)
    return Counter(words)

def cosine_sim(sentence1, sentence2):
    vec1 = text_to_vector(sentence1)
    vec2 = text_to_vector(sentence2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
    

In [39]:
ranks = []
score_all = []
ids = []
def find_answers():
    http = urllib3.PoolManager()
    tree = ET.parse('validation.xml')
    root = tree.getroot()
    
    
    for q in root.findall('Question'):
        ans_urls = []
        ans_texts = []
        question = q.find('QuestionText').text
        att = q.attrib['QID']
        ids.append(att)
        print(att)
        li = q.find('AnswerList')
        for ans_li in li.findall('Answer'):
            url = ans_li.find('AnswerURL').text
            ans_urls.append(url)
            text = ans_li.find('AnswerText').text
            ans_texts.append(text)
        
        contexts = []
        count =0
        '''
        for url in ans_urls:    
            if(url == '#'):
                contexts.append(ans_texts[count])
            else:
                contexts.append(extract_context(url))
            count = count + 1
        '''
        ans = []
        ans_scores = []
        
        for i in range(len(ans_texts)):
            ans_cont = model([ans_texts[i]], [question])
            ans.append(ans_cont[0][0])
            ans_scores.append(ans_cont[2][0])
        
        '''
        score_dict = dict()
        for i in range(len(ans_scores)):
            score_dict[i] = ans_scores[i]
        a = sorted(score_dict.items(), key = lambda score_dict:(score_dict[1], score_dict[0]))
        print(a)
        ## Later ##
        '''
        sorted_scores = sorted(ans_scores)
        new_sorted_scores = []
        for i in range(len(sorted_scores)):
            if(i==0 or sorted_scores[i]!= new_sorted_scores[len(new_sorted_scores)-1]):
                new_sorted_scores.append(sorted_scores[i])
        ans_ranks = [[] for i in range(len(new_sorted_scores))]
        for cont in range(len(ans_texts)):
            indexval = new_sorted_scores.index(ans_scores[cont])
            ans_ranks[indexval].append(cont)
      
        final_ranks = []
        for i in range(len(ans_ranks)):
            if(len(ans_ranks[i])>1):
                cos_val = dict()
                for val in range(len(ans_ranks[i])):
                    cosine = cosine_sim(ans[ans_ranks[i][val]],ans_texts[ans_ranks[i][val]])
                    cos_val[ans_ranks[i][val]]=cosine
                
                a = sorted(cos_val.items(), key = lambda cos_val:(cos_val[1], cos_val[0]))
                for i in range(len(a)) :
                    final_ranks.append(a[i][0])
            else:
                final_ranks.append(ans_ranks[i][0])
       
        ranks.append(final_ranks)
        score_all.append(ans_scores)
    print(ranks)

find_answers()

2
3
4
9
11
12
17
18
20
23
24
36
38
40
44
48
53
54
56
59
63
66
68
71
73
[[8, 4, 0, 6, 7, 5, 1, 3, 2, 9], [0, 5, 9, 6, 7, 1, 8, 2, 3, 4], [0, 8, 6, 5, 4, 1, 3, 2, 7], [0, 7, 9, 5, 8, 1, 6, 3, 4, 2], [1, 7, 5, 8, 6, 0, 3, 4, 2], [7, 9, 3, 2, 8, 5, 6, 0, 1, 4], [8, 5, 6, 4, 0, 7, 3, 2, 9, 1], [5, 7, 1, 9, 8, 0, 3, 2, 6, 4], [1, 6, 2, 8, 9, 4, 10, 5, 7, 0, 3], [1, 9, 7, 6, 2, 5, 3, 4, 0, 8], [3, 7, 5, 2, 4, 6, 9, 8, 0, 1], [1, 10, 3, 5, 7, 2, 9, 0, 4, 8, 6], [7, 5, 6, 1, 3, 8, 4, 2, 9, 0], [7, 6, 5, 4, 8, 2, 3, 1, 0], [6, 8, 4, 1, 7, 5, 3, 0, 2], [1, 0, 2], [0, 1, 7, 2, 3, 5, 4, 6], [1, 4, 5, 3, 2, 7, 0, 6, 8], [5, 2, 1, 3, 0, 4], [8, 7, 4, 1, 5, 2, 9, 3, 6, 0], [9, 8, 0, 4, 6, 7, 3, 1, 5, 2], [6, 7, 9, 10, 0, 1, 8, 2, 3, 4, 5], [6, 5, 4, 2, 1, 10, 8, 3, 9, 0, 7], [1, 2, 3, 5, 4, 0, 6], [9, 5, 10, 4, 7, 8, 0, 1, 6, 3, 2]]


In [49]:
subm_array = []
for i in range(len(ranks)):
    for j in range(len(ranks[i])):
        arr = []
        arr.append(str(ids[i]))
        arr.append(str(ids[i])+"_Answer"+str(ranks[i][len(ranks[i])-j-1]+1))
        if(score_all[i][ranks[i][len(ranks[i])-j-1]] > 5000):
            arr.append(1)
        else:
            arr.append(0)
        subm_array.append(arr)
        
    
                

In [50]:
import pandas as pd
df = pd.DataFrame(subm_array)

In [51]:
df

,0,1,2
0,2,2_Answer10,1
1,2,2_Answer3,1
2,2,2_Answer4,1
3,2,2_Answer2,1
4,2,2_Answer6,1
5,2,2_Answer8,1
6,2,2_Answer7,0
7,2,2_Answer1,0
8,2,2_Answer5,0
9,2,2_Answer9,0


In [52]:
df.to_csv('out.csv', encoding='utf-8')

In [45]:
trues = []
def find_answers():
    http = urllib3.PoolManager()
    tree = ET.parse('validation.xml')
    root = tree.getroot()
    
    
    for q in root.findall('Question'):
        ans_urls = []
        ans_texts = []
        question = q.find('QuestionText').text
        att = q.attrib['QID']
        li = q.find('AnswerList')
        temp = [[] for i in range(len(li.findall('Answer')))]
        count = 0 
        for ans_li in li.findall('Answer'):
            count =count+1
            true = []
            att1 = ans_li.attrib['ReferenceRank']
            att2 = ans_li.attrib['ReferenceScore']
            true.append("Q"+str(att))
            true.append("A"+str(count))
            if(att2=="4" or att2=="3"):
                true.append(1)
            else:
                true.append(0)
            temp[int(att1)-1] = true
            
        for i in temp:
            trues.append(i)
            
find_answers()        
trues

[['Q2', 'A6', 1],
 ['Q2', 'A10', 1],
 ['Q2', 'A1', 1],
 ['Q2', 'A3', 1],
 ['Q2', 'A8', 0],
 ['Q2', 'A7', 0],
 ['Q2', 'A2', 0],
 ['Q2', 'A4', 0],
 ['Q2', 'A5', 0],
 ['Q2', 'A9', 0],
 ['Q3', 'A1', 1],
 ['Q3', 'A4', 1],
 ['Q3', 'A5', 1],
 ['Q3', 'A8', 1],
 ['Q3', 'A6', 1],
 ['Q3', 'A3', 0],
 ['Q3', 'A7', 0],
 ['Q3', 'A9', 0],
 ['Q3', 'A10', 0],
 ['Q3', 'A2', 0],
 ['Q4', 'A1', 1],
 ['Q4', 'A3', 1],
 ['Q4', 'A2', 0],
 ['Q4', 'A5', 0],
 ['Q4', 'A4', 0],
 ['Q4', 'A6', 0],
 ['Q4', 'A7', 0],
 ['Q4', 'A8', 0],
 ['Q4', 'A9', 0],
 ['Q9', 'A2', 1],
 ['Q9', 'A1', 1],
 ['Q9', 'A3', 1],
 ['Q9', 'A5', 1],
 ['Q9', 'A7', 0],
 ['Q9', 'A6', 0],
 ['Q9', 'A4', 0],
 ['Q9', 'A10', 0],
 ['Q9', 'A8', 0],
 ['Q9', 'A9', 0],
 ['Q11', 'A2', 1],
 ['Q11', 'A1', 1],
 ['Q11', 'A3', 0],
 ['Q11', 'A4', 0],
 ['Q11', 'A5', 0],
 ['Q11', 'A6', 0],
 ['Q11', 'A7', 0],
 ['Q11', 'A8', 0],
 ['Q11', 'A9', 0],
 ['Q12', 'A8', 1],
 ['Q12', 'A3', 1],
 ['Q12', 'A1', 1],
 ['Q12', 'A5', 1],
 ['Q12', 'A7', 0],
 ['Q12', 'A6', 0],
 ['Q12', '

In [46]:
df1 = pd.DataFrame(trues)

In [48]:
df1.to_csv('true_out.csv', encoding='utf-8')